# Пайплайн запуска модели

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

Токенайзер грузим тот же, а вот веса самой модели берем новые уже предобученные.

In [17]:
tokenizer = GPT2Tokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
model = GPT2LMHeadModel.from_pretrained("model_ru2/checkpoint-38019").cuda()
# model = GPT2LMHeadModel.from_pretrained("model_ru/checkpoint-31165").cuda()

Функция генерации принимает в себя категорию и рейтинг. Также дополнительно задается температура модели, что позволяет регулировать ее "креативность". Из интересного в этой функции доработано обрезание под нашу задачу - как мы можем помнить, наши отзывы тем короче, чем выше оценка, соответственно имеет смысл обрезать раньше сгенерированный текст для положительных отзывов. 

In [66]:
def generate_review(category, rating, temperature):
    category_str = 'Категории: ' + ', '.join(category.split(';'))
    rating_to_quality = {
        1: 'Очень негативный отзыв',
        2: 'Негативный отзыв',
        3: 'Обычный отзыв',
        4: 'Положительный отзыв',
        5: 'Очень положительный отзыв'
    }

    input_text = f"{category_str}. {rating_to_quality[rating]}:"

    input_ids = tokenizer.encode(input_text, return_tensors="pt").cuda()

    # Генерация текста
    with torch.no_grad():
        output = model.generate(input_ids, max_length=64 + (5 - rating) * 32, num_return_sequences=1, temperature=temperature, do_sample=True)
            
    # Декодирование и вывод текста
    review = tokenizer.decode(output[0], clean_up_tokenization_spaces=True)
    review = review[len(input_text) + 1:][:(-review[::-1].find('.'))]
    return review

Тут далее разные тесты

In [68]:
generate_review("Парикмахреская;Барбер шоп", 1, 0.7)

'Написал отзыв о работе салона красоты, потому что, когда у меня была запись по телефону, я не знал,  что будет стрижка, а когда приехал, оказалось, что девушка не знает где и когда мне её нужно, я не знал.  По телефону мне сказали, что мне надо мне "всё, просто уточните, я уже записан, но когда пришёл и тк у меня только в записан, мне сказали, что надо в итоге, и вообще есть уже, что она будет. Я в итоге я бы не знал когда мне надо, я решил, что мне надо было.  "а мне некогда" - сказал я, и мне сказали, что мне нужно у вас записаться.'

In [72]:
[generate_review("Супермаркет;Продукты;Пятерочка", 5, 0.7) for i in range(4)]

['Очень нравится сеть Пятерочка. Всегда покупаю здесь продукты первой необходимости, т.к. там всегда всё свежее и вкусное. Часто захожу, когда нет времени просто ходить в магазин.',
 'Магазин шаговой доступности. На этом принципе он и держится. Удобно расположен. Свежий рыбный отдел, всегда свежая рыба, свежая молочная продукция в ассортименте.  Много готовой еды.',
 'Магазин хороший и персонал хороший,  и ассортимент продукции хороший. Но. Вчера зашла купить что-то от души.  Хотелось бы сказать пару слов.  Хотелось бы отметить  о . 1.  1.',
 'Добрый день. Сегодня мы с дочкой посетили ваш магазин. Он нам очень понравился. В магазине чисто, очень много товара. Особенно в отделе овощных продуктов. Цены хорошие, большой выбор.']

In [73]:
[generate_review("Супермаркет;Продукты;Пятерочка", 3, 0.7) for i in range(4)]

['Магазин как магазин, как и все в этом городе, но в основном. Выбор товаров очень велик. Но вот качество оставляет желать лучшего. В магазине очень много просрочки. В последний раз, из за этого несколько раз была не довольна.  На кассе кассирша не довольна, что я попросила оплатить по карте, которая была в чеке, но у меня уже не срабатывает терминал, так как я не смогла мне товар и мне пробила не тот товар, который я просила.',
 'В целом магазин очень неплохой, есть все для быта, но вот то что не устраивает по ассортименту, я не могу сказать.. Я люблю в плане чистоты, то что я могу сказать, что в магазине есть.. Персонал хороший.. 1. Не понятно, почему на кассе работают 2 человека, одна продавец, сидит на кассе, а вторая за прилавком, а вторая за кассой.. 2. На днях кассир (я так понимаю, это я)..',
 'Хороший магазин, но вот продавцы часто меняются. Один раз зашла в магазин, чтобы купить выпечку, а мне в кассе отказали из-за того, что я не хотела очередь. Я конечно не продала товар, к

In [74]:
[generate_review("Супермаркет;Продукты;Пятерочка", 1, 0.7) for i in range(4)]

['Отвратительный магазин. Не советую. Я брал творог в магазине «Магнит», на кассе он оказался с плесенью и плесенью. Вкусные были только сырники и творожные сырники, на мой взгляд. На кассе пробили сырники с творогом, которые я съел только 3 сыра и творожные творожные, которые я не попробовал. Не знаю, а именно сырники, а не творожные сырники и творожные. При этом все остальные, которые я не попробовал. В итоге они были с плесенью. И на мой вопрос «Откуда плесень?» продавец сказала: «Это только в творожных сырниках, это плесень, а не в творожных». Тут я еще раз посмотрел на срок годности, а он оказался просрочен.',
 'Вчера заехала в магазин,чтобы купить молочные продукты.Оказалось,что молочки по акции  не купить.Продавец сказала,что молочка по акции у них на 50 рублей дешевле.Пошла за молоком и увидела,что молочка.У меня была скидка-этот молочный магазин для меня ни о чем.Продавец была в пятерочке.Я говорю,что акция у них называется "молока там нет,а акция у них "Ассортимент молочка дл